# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

from sql_queries import *
from functions import *

In [2]:
# checking current working directory
pwd = os.getcwd()

# calling preprocessing function
preprocessing_csv_file(pwd)

# check the number of rows in new csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [3]:
from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])
session = cluster.connect()

#### Create Keyspace

In [4]:
try:
    session.execute(create_keyspace)
except Exception as e:
    print(e)

#### Set Keyspace

In [5]:
try:
    session.set_keyspace('sparkifydb')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [6]:
session.execute(session_table_drop)

try:
    session.execute(session_table_create)
except Exception as e:
    print(e)                   
                    

In [7]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f, delimiter=',')
    next(csvreader) 
    for line in csvreader:
        session.execute(session_table_insert,
                   (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))       

#### Do a SELECT to verify that the data have been inserted into each table

In [8]:
# SELECT statement with specifications made in query example above
try:
    rows = session.execute(session_table_select, (338, 4))
except Exception as e:
    print(e)
    
result = list(rows)
data = []
for row in result:
    data.append(row)
    df = pd.DataFrame(data)
df

,artist,song,length
0,Faithless,Music Matters (Mark Knight Dub),495.307312


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [11]:
session.execute(user_table_drop)

try:
    session.execute(user_table_create)
except Exception as e:
    print(e)
    
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        session.execute(user_table_insert, 
                        (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[0], line[4]))
        
# SELECT statement with specifications made in query example above
try:
    rows = session.execute(user_table_select, (10, 182))
except Exception as e:
    print(e)
    
result = list(rows)
data = []
for row in result:
    data.append(row)
    df = pd.DataFrame(data)
df

,artist,song,iteminsession,firstname,lastname
0,Down To The Bone,Keep On Keepin' On,0,Down To The Bone,Cruz
1,Three Drives,Greece 2000,1,Three Drives,Cruz
2,Sebastien Tellier,Kilometer,2,Sebastien Tellier,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,3,Lonnie Gordon,Cruz


In [12]:
session.execute(song_table_drop)

try:
    session.execute(song_table_create)
except Exception as e:
    print(e)
    
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        session.execute(song_table_insert, 
                        (line[9], int(line[10]), line[10], line[9]))
        
# SELECT statement with specifications made in query example above
try:
    rows = session.execute(song_table_select, ('All Hands Against His Own',))
except Exception as e:
    print(e)
    
result = list(rows)
data = []
for row in result:
    data.append(row)
    df = pd.DataFrame(data)
df

,firstname,lastname,userid,song
0,29,All Hands Against His Own,29,All Hands Against His Own
1,80,All Hands Against His Own,80,All Hands Against His Own
2,95,All Hands Against His Own,95,All Hands Against His Own


### Drop the tables before closing out the sessions

In [13]:
session.execute(session_table_drop)
session.execute(user_table_drop)
session.execute(song_table_drop)

### Close the session and cluster connection¶

In [14]:
session.shutdown()
cluster.shutdown()